In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
from AIToolbox.torchtrain.train_loop import TrainLoop, TrainLoopModelCheckpointEndSave
from AIToolbox.torchtrain.model import TTModel
from AIToolbox.torchtrain.callbacks.performance_eval_callbacks import ModelPerformanceEvaluation, ModelPerformancePrintReport
from AIToolbox.experiment.result_package.basic_packages import ClassificationResultPackage

## Define the model architecture

In [6]:
class Net(TTModel):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

    def get_loss(self, batch_data, criterion, device):
        data, target = batch_data
        data, target = data.to(device), target.to(device)

        output = model(data)
        loss = criterion(output, target)

        return loss

    def get_predictions(self, batch_data, device):
        data, y_test = batch_data
        data = data.to(device)

        output = model(data)
        y_pred = output.argmax(dim=1, keepdim=False)  # get the index of the max log-probability

        return y_pred.cpu(), y_test, {}

## Define data loaders

In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [8]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

0it [00:00, ?it/s]

9920512it [00:01, 4999763.67it/s]                             


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 94965.40it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


1654784it [00:01, 1447887.15it/s]                            
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


8192it [00:00, 36808.22it/s]            

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [9]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

## Init the model and decide on optimizer and the loss

In [10]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
criterion = F.nll_loss

In [11]:
callbacks = [ModelPerformanceEvaluation(ClassificationResultPackage(), {'batch_size': 100, 'lr': 0.001},
                                        on_train_data=True, on_val_data=True),
             ModelPerformancePrintReport(['train_Accuracy', 'val_Accuracy'], strict_metric_reporting=True)]

## Train the model

In [ ]:
TrainLoop(model,
          train_loader, test_loader, None,
          optimizer, criterion)(num_epoch=3, callbacks=callbacks)

## Experiment tracking training

In [25]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
criterion = F.nll_loss

In [26]:
callbacks = [ModelPerformanceEvaluation(ClassificationResultPackage(), {'batch_size': 100, 'lr': 0.001},
                                        on_train_data=True, on_val_data=True),
             ModelPerformancePrintReport(['train_Accuracy', 'val_Accuracy'], strict_metric_reporting=True)]

In [ ]:
TrainLoopModelCheckpointEndSave(model,
                                train_loader, test_loader, test_loader,
                                optimizer, criterion,
                                project_name='localRunCNNTest_uni_model',
                                experiment_name='CNN_MNIST_test',
                                local_model_result_folder_path='model_results',
                                hyperparams=args.__dict__,
                                test_result_package=ClassificationResultPackage(),
                                cloud_save_mode=None)(num_epoch=5, callbacks=callbacks)